In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-05-28"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
9268,2024-05-28,Israel Superliga,15:10,Hapoel Jerusalem,Hapoel Holon,1.38,3.00,159.5,1.90,1.88,0.0,0.00,0.00,jVdOck5j,0.724638,0.333333,0.526316,0.531915,0.057971,139.878,24.782399,0.177172,2.4,1.341641,0.559017,166.46,1.610,0.286793,0.178132,49.0,179.118,37.851308,0.211321,1.8,1.643168,0.912871,216.60,2.374,0.753346,0.317332,-26.0,82,76,2.03,2.85,127.600,163.278,0.523065,0.007483,NaN,-0.31,-0.062,-6.129032,0.738796,0.7,-0.038796,-1.33,-0.266,-7.518797,0.493233,0.4,-0.093233
9269,2024-05-28,México Cibacopa,00:15,Halcones de Obregon,Rayos de Hermosillo,1.90,1.82,178.5,1.83,1.85,0.0,0.00,0.00,U3UKRDr2,0.526316,0.549451,0.546448,0.540541,0.075766,258.958,64.079820,0.247453,0.6,1.341641,2.236068,240.26,2.882,0.723927,0.251189,-41.0,128.208,19.238123,0.150054,2.4,1.341641,0.559017,100.01,1.430,0.113798,0.079579,44.0,82,73,2.93,1.37,157.404,170.674,0.030413,0.007686,NaN,0.79,0.158,5.696203,0.601278,0.7,0.098722,0.36,0.072,11.388889,0.514674,0.6,0.085326
9270,2024-05-28,México Cibacopa,00:15,Ostioneros de Guaymas,Zonkeys de Tijuana,2.76,1.41,172.5,1.86,1.84,0.0,0.00,0.00,AwWWxvi3,0.362319,0.709220,0.537634,0.543478,0.071539,268.214,65.612496,0.244627,1.8,1.643168,0.912871,294.06,3.056,0.891308,0.291658,33.0,121.948,37.707780,0.309212,1.8,1.643168,0.912871,81.84,1.286,0.149432,0.116199,33.0,78,66,3.77,1.24,185.536,169.576,0.457839,0.007644,NaN,-3.36,-0.672,-2.619048,0.498447,0.4,-0.098447,-0.23,-0.046,-8.913043,0.519289,0.5,-0.019289
9271,2024-05-28,México Cibacopa,00:15,Pioneros de Los Mochis,Astros,3.08,1.34,188.5,1.87,1.83,0.0,0.00,0.00,jFzP9uNG,0.324675,0.746269,0.534759,0.546448,0.070944,314.936,139.404093,0.442643,1.2,1.643168,1.369306,489.30,3.302,1.241398,0.375953,-26.0,129.968,8.787922,0.067616,2.4,1.341641,0.559017,118.17,1.224,0.055045,0.044972,40.0,105,101,4.66,1.17,177.040,140.168,0.556727,0.015289,NaN,-0.74,-0.148,-14.054054,0.563452,0.5,-0.063452,-0.75,-0.150,-2.266667,0.687219,0.7,0.012781
9272,2024-05-28,México Cibacopa,00:15,Venados de Mazatlan,Angeles CDMX,1.74,1.99,181.5,1.83,1.87,0.0,0.00,0.00,dz6cOvxA,0.574713,0.502513,0.546448,0.534759,0.077225,184.466,62.684312,0.339815,0.6,1.341641,2.236068,258.44,2.018,0.717300,0.355451,-18.0,157.730,52.053138,0.330014,2.4,1.341641,0.559017,127.88,1.492,0.357379,0.239530,21.0,91,92,2.84,1.39,162.346,242.240,0.094786,0.015289,NaN,0.92,0.184,4.021739,0.609429,0.6,-0.009429,1.60,0.320,3.093750,0.542861,0.7,0.157139
9273,2024-05-28,Países Baixos Dbl,15:00,Den Bosch,Leiden,1.32,3.20,152.5,1.80,1.86,-7.5,1.92,6.50,jJ0c9b6U,0.757576,0.312500,0.555556,0.537634,0.070076,107.510,10.844376,0.100869,1.8,1.643168,0.912871,106.20,1.280,0.234521,0.183219,39.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,90,64,1.18,2.37,0.000,0.000,0.588213,0.023184,0.769252,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
9274,2024-05-28,Rússia Liga Vtb United,13:30,Unics Kazan,CSKA Moscow,1.80,2.02,165.5,1.84,1.92,-2.5,2.05,2.29,0fyOaO2B,0.555556,0.495050,0.543478,0.520833,0.050605,163.114,30.386517,0.186290,1.8,1.643168,0.912871,160.55,1.878,0.383888,0.204413,18.0,161.614,51.829257,0.320698,1.8,1.643168,0.912871,210.24,1.868,0.358078,0.191691,11.0,95,96,1.69,2.19,129.076,184.272,0.081447,0.030090,0.078205,-0.42,-0.084,-9.523810,0.710140,0.7

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
9274,13:30,Rússia Liga Vtb United,Unics Kazan,CSKA Moscow,1.8,Back Home
